<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# 為什麼需要裝飾詞 decorator？
1. 降低程式碼重複率
2. 易讀性高
3. 靈活度高

In [1]:
def foo():
    return 'bar'

print(foo)
print(foo())

<function foo at 0x000001FB9F54DE50>
bar


## A general example - add a timer into any general function.  
首先我們在timer將 func 變數名稱傳入; 接下來定義 def wrap(sleep_time) 函式; 並且在裡面將剛剛傳入的 func(sleep_time) 調用

In [38]:
import time
 
def timer(func): #func is the original function.
    def wrap(sleep_time):
        t_start = time.time()
        func(sleep_time) #call the originial functiion
        t_end = time.time()
        t_count = t_end - t_start
        print('[花費時間]', t_count)
    return wrap

In [39]:
def dosomething(sleep_time):
    print('do some thing')
    time.sleep(sleep_time)

foo = timer(dosomething)(3)

do some thing
[花費時間] 3.0019407272338867


## 語法糖 (Syntax Candy)
直接加上 @timer，並直接調用 dosomething() 函式執行

In [15]:
@timer #add timer function into dosomethin directly
def dosomething(sleep_time):
    print('do some thing')
    time.sleep(sleep_time)

dosomething(3)

do some thing
[花費時間] 3.000098705291748


## 裝飾詞的特性 (副作用)：函式名稱
裝飾詞在被 wrap 包一層後，其 __name__ 屬性就會被修改成 wrap

In [16]:
print(dosomething.__name__)

wrap


使用 python 內建的 functools，只需要在 def wrap()之前，加上 @wraps(func)，即可獲得原先的 __name__ 屬性 dosomething

In [21]:
from functools import wraps
 
def timer(func):
    @wraps(func) #recover the name of the functiion
    def wrap():
        t_start = time.time()
        func()
        t_count = time.time() - t_start
        print('[花費時間]', t_count)
    return wrap

@timer
def dosomething():
    print('do some thing')

dosomething() # = timer(dosomething)
print(dosomething.__name__)

do some thing
[花費時間] 0.0
dosomething


In [30]:
from functools import wraps
def timer(func):
    @wraps(func)
    def wrap():
        print('this is timer.')
        t_start = time.time()
        func()
        t_end = time.time()
        t_count = t_end - t_start
        print('[花費時間]', t_count)
    return wrap
 
def func_print_one(func):
    def wrap():
        print('this is func_print_one')
        func()
    return wrap
 
def func_print_two(func):
    def wrap():
        print('this is func_print_two')
        func()
    return wrap
 
@timer
@func_print_one
@func_print_two
def dosomething():
    print('do some thing')

dosomething()

this is timer.
this is func_print_one
this is func_print_two
do some thing
[花費時間] 0.0


## 裝飾需要參數的函式

In [37]:
import time
def timer(func):
    def wrapper(*args, **kwargs):
        print('start...')
        t1 = time.perf_counter()
        func(*args, **kwargs)
        print('Elaspsed time:', time.perf_counter()-t1)
    return wrapper

@timer
def works(start, stop):
    total = 0
    for i in range(start, stop):
        total += i
    print(total)
    
works(1,1000)

start...
499500
Elaspsed time: 2.5600002118153498e-05


## 再加上裝飾詞的參數 (\*args, \**kargs)
只需在 def wrapper() 和 func() 中加入 *args, **kargs 即可調用參數

In [42]:
def repeat(num):
    """帶參數的裝飾器"""
    def my_decorator(func):
        def wrapper(*args, **kwargs):
            for i in range(num):
                print(f"Running {func.__name__}")
                func(*args, **kwargs)
        return wrapper
    return my_decorator

@repeat(num=2)  # 使用帶有參數的裝飾器
def say_hello(name):
    print(f"Hello {name}")

say_hello("Alice") # => repeat(num=2)(say_hello('Alice'))

Running say_hello
Hello Alice
Running say_hello
Hello Alice


## class的裝飾器
將 wrap 寫在 __call__ 裡面來調用

In [56]:
import time
class timer:
    def __init__(self, time_sleep):
        print('--init--', f'{time_sleep=}')
        self.time_sleep = time_sleep
    
    def __call__(self, func):
        def wrapper(*args, **kwargs):
            print('start...')
            t1 = time.perf_counter()
            time.sleep(self.time_sleep)
            value = func(*args, **kwargs) #value = None            
            print('Elaspsed time:', time.perf_counter()-t1)
            return value
        return wrapper

@timer(time_sleep=3)
def dosomethingClass(a, b):
    print('do some thing')
    print('a + b = ', a + b)
    
dosomethingClass(1, 2)

--init-- time_sleep=3
start...
do some thing
a + b =  3
Elaspsed time: 3.0045147000018915


In [ ]:
#https://ithelp.ithome.com.tw/articles/10200763
#https://www.maxlist.xyz/2019/12/07/python-decorator/